
## In this notebook, you will see all the steps sequentially performed to be able to utilize the complete functionality of OAB framework. The steps are as follows :
0. SETUP
1. DATA
2. DATA SELECTION
3. PREPROCESSING
4. SAMPLING
5. ALGORITHM TRAINING AND TESTING
6. EVALUATION
7. SHOW BENCHMARK RESULTS
8. REPRODUCIBILTY
9. EXTENDING THE BENCHMARK(with own Algorithm)

This notebook focuses on <b>Unsupervised Tabular Data</b>. Let's begin!

# **0. SETUP**

`oab` framework can be integrated in your Python environment  as a PyPi package  using the following command:

In [ ]:
#ID 1(0)

#%%capture
# pip install oab
!pip install example-pkg-jd-kiel --extra-index-url=https://test.pypi.org/simple/

`Cloning` the repository:

oab is an open-source framework which can be accessed at https://github.com/ISDM-CAU-Kiel/oab. To use this .ipynb notebook successfully, the formerly mentioned repository needs to be cloned with the following command and this notebook must be run(if this is not the case already) within the cloned repository from the path:

<b>/oab/Tutorials/Unsupervised_Anomaly_Detection_on_Benchmark_Tabular_Data.ipynb</b>

In [ ]:
#ID 2(0)
!git clone https://github.com/ISDM-CAU-Kiel/oab.git

Now importing the necessary functions and inernal variables

In [ ]:
#ID 3(0)



import sys
import os
from datetime import datetime 
from pathlib import Path
#sys.path.append('../..')            #setting the current directory path
sys.path.insert(0,f"{Path(os.getcwd()).parent}") # setting the 
%load_ext autoreload
%autoreload 2


#other  necessary imports
from oab.data.unsupervised import UnsupervisedAnomalyDataset
from oab.data.load_dataset import load_dataset
from oab.data.load_recipe_functions import *




#from oab.data.datasets import *

# necessary imports for algorithm comparisons and defining seeds
from oab.evaluation import EvaluationObject, ComparisonObject,all_metrics
import random
import numpy as np
import tensorflow as tf
import torch

## **0.1 NOTEBOOK AND CELL STRUCTURE** 

In this notebook there are certain sections where the user s required to enter its own information which are marked as comments of the form :

<b>### ADD YOUR CODE ###</b>  , so <b>###</b> can be searched to know what are those sections.

All cells are assigned an ID, as a comment at the top of the cell,for example as: <b>#ID 10(5)</b>, where 10 denotes the cell ID and 5 denotes the Section.

## 0.2 DETAILS OF THIS BENCHMARK RUN

In [3]:
#ID 4(0)
### ADD YOUR BENCHMARK NAME HERE ###
benchmark_name="Paper_A" 


datasets_info_path=Path(os.getcwd()).parent/"oab"/"data"/"datasets.yaml" # getting path to "datasets.yaml" which contains information about all tabular datasets

recipes_parent_path=Path(os.getcwd()).parent/"notebooks"/"benchmark_tabular"
dataset_folder=recipes_parent_path/"datasets" # all dataset-folders are contained in this folder
#print(dataset_folder)

benchmark_type="ust"     # benchmark run for unsupervised tabular datasets(sst)
if not os.path.exists(recipes_parent_path/benchmark_name): #creating directory for this benchhmark for storing recipes
    os.makedirs(recipes_parent_path/benchmark_name)

    
time=datetime.now().strftime("%Y%m%d%H%M%S") # timestamp set for this run  
new_recipe_path=f"{recipes_parent_path}/{benchmark_name}/{time}-{benchmark_name}-{benchmark_type}-recipe.yaml" # recipe path for new recipe created in this run   
print(f"{time}-{benchmark_name}-{benchmark_type}-recipe.yaml")

20211217002028-Paper_A-ust-recipe.yaml


### For reproducing a previously-created recipe without adding new datasets and algorithms from this benchmark  ,  skip to :

### `#ID 20(5)`

# **1. DATA**

First of all, we will have a look at the Datasets that are pre-installed in OAB which can be used for benchmarking

In [4]:
# ID 5(1)
for i in get_tabular_dataset_names():
    print(f"{i[0]}.{i[1]}")



0.NASA_ground_data
1.annthyroid
2.arrhytmia
3.boston
4.breastw
5.cardio
6.forest_cover
7.http
8.ionosphere
9.mammography
10.musk
11.myTabularDataset
12.myTabularDataset2
13.optdigits
14.page-blocks
15.pendigits
16.pima
17.pulsar_star
18.shuttle
19.smtp
20.spambase
21.thyroid
22.vertebral
23.wilt
24.wine



`oab` provides a variety of tabular datasets that can easily be loaded, 

`1.` If a user is interested in using her own tabular dataset, the following steps have to be followed: 

 (a) Ensure that `own` dataset(s) information is stored in the file `datasets.yaml` which is located at
<b>"/oab/data"</b>(by executon of  #ID 7(1))
 
 
 (b) The `folder` containing the dataset must be stored in `datasets` folder of the OAB and name should be same as the dataset_name
 
 (c) Then, own dataset(s) can be loaded just like the pre-installed OAB datasets 
 
 `2.` If user's dataset is provided **via a URL**, then it would be downloaded and stored in the OAB's "datasets" folder.
 
 The files in the `folder` that are downloaded or stored manually in "datasets" folder , can be of variety of formats such as:
 
 1. 'csv           
 2. 'zip'
 3.'gz_single_file'
 4.'mat'
 5.'mat_old
 
 If user has her dataset file in one of these formats, or has multiple files, then `oab` automatically makes one file out of that which is then input to the oab. We will see the case here, when user loads her dataset through local `folder directory`.

Here's the structure of how the datasets' information are stored in "datasets.yaml".

In [5]:
# ID 6(1)
!cat {datasets_info_path}

NASA_ground_data:
  class_labels: last
  credits: Sayyad Shirabad, J. and Menzies, T.J. (2005) The PROMISE Repository of
    Software Engineering Databases. School of Information Technology and Engineering,
    University of Ottawa, Canada.
  dataset_format: csv
  destination_filenames: [NASA_ground_data.csv]
  destination_yaml: NASA_ground_data_preprocessing.yaml
  filename_in_folder: NASA_ground_data.csv
  filenames_to_concatenate: null
  foldername: NASA_ground_data
  load_csv_arguments: {header: 0}
  name: NASA_ground_data
  short_description: null
  url_yaml: https://raw.githubusercontent.com/jandeller/test/main/test6.yaml
  urls_dataset: ['https://www.openml.org/data/get_csv/53950']
annthyroid:
  class_labels: last
  dataset_format: mat_old
  destination_filenames: [annthyroid.mat]
  filename_in_folder: annthyroid.mat_old
  filenames_to_concatenate: null
  foldername: annthyroid
  load_csv_arguments: {header: null}
  name: annthyroid
  urls_dataset: ['https://www.dropbox.com/s/ai

In [6]:
#ID 7(1)


### ADD OWN DATASET(S) DETAILS ###   see #ID 3(1) for exact parameters which are to be entered

own_datasets_info= [{
                        'dataset_name':'myTabularDataset',
                    }]


 # 'myTabularDataset' is the name of the Dataset(which the user loads for benchmarking)
### Add more dictionaries to the list `own_datasets_info` with datasets information like example below
                   #           { 
                   #               'dataset_name':'XYZDataset'
                   #               'filenames_to_concatenate':['train_example.csv','test_example.csv']
                   #               ....
                   #            }
                
  



### ADD DATASETNAME(S) FROM OAB'S DATASETS HERE ###

benchmark_datasets_list=['spambase']   # More of OAB's datasets can be added to this list




## contains dataset objects of own_datasets_list as well as benchmark_datasets_list   
datasets={}


# Adding and Loading own datasets
for dataset_details in own_datasets_info:
     
     datasets[dataset_details['dataset_name']]=load_own_tabular_dataset(**dataset_details)
     print("Dataset added and loaded!!!")

print(datasets)


Dataset added and loaded!!!
{'myTabularDataset': <oab.data.classification_dataset.ClassificationDataset object at 0x7fc24009ff90>}


# **2. DATA SELECTION**


Datasets can either be loaded directly as anomaly datasets or as classification datasets. In the former case, the dataset is automatically fully prepared and ready for sampling. In the latter case, further preprocessing is still possible and necessary.



**After adding and loading own dataset(s) in #ID 7(1) and now the user is able to select other benchmarking datasets :**

Now,  we'll have a look at all the datasets again which are pre-installed in OAB, so that they can be chosen for the benchmark run.

In [7]:
#ID 8(2)


for i in get_tabular_dataset_names():
    print(f"{i[0]}.{i[1]}")


0.NASA_ground_data
1.annthyroid
2.arrhytmia
3.boston
4.breastw
5.cardio
6.forest_cover
7.http
8.ionosphere
9.mammography
10.musk
11.myTabularDataset
12.myTabularDataset2
13.optdigits
14.page-blocks
15.pendigits
16.pima
17.pulsar_star
18.shuttle
19.smtp
20.spambase
21.thyroid
22.vertebral
23.wilt
24.wine


### **2.1 Load anomaly detection datasets (with or without further preprocessing)**

In this section, we load some pre-installed data sets. This can be achieved using the `load_dataset` function. By default, it creates an anomaly dataset from which sampling is directly possible  but we can first create classsifcation dataset and then anomaly dataset,either with the preprocessing applied (`preprocess_classification_dataset=True`) i.e. standard or custom operations like treat_missing_values,delete_columns,etc. are performed, or without (`preprocess_classification_dataset=False`, default).

`In our case` we set have already imported own datasets with `anomaly_dataset=False ` and `preprocess_classification_dataset=False` in <b>#ID 7(1)</b> and we will also load the OAB datasets in the same way in <b>#ID 9(2)</b>

Note that as discussed in the paper, multiclass classification datasets like `spambase` and `annthyroid` are loaded with the class label `0` as normal label and all other labels as anomaly labels by default. (Alternatively, `oab` can automatically iterate through all classes as normal classes. This is not covered here.

In [8]:
#ID 9(2)


for dataset_name in benchmark_datasets_list:  # loading benchmark's datasets
    datasets[dataset_name]=load_dataset(dataset_name,anomaly_dataset=False,preprocess_classification_dataset=False,dataset_folder=dataset_folder)


Credits: Dua, D. and Graff, C. (2019). UCI Machine Learning Repository [http://archive.ics.uci.edu/ml]. Irvine, CA: University of California, School of Information and Computer Science.


In [9]:
#ID 10(2)

print(datasets)

{'myTabularDataset': <oab.data.classification_dataset.ClassificationDataset object at 0x7fc24009ff90>, 'spambase': <oab.data.classification_dataset.ClassificationDataset object at 0x7fc19c566cd0>}


# **3. PREPROCESSING**

Standard preprocessing steps like deleting columns, encoding categorical values differently, or removing missing values can be performed  to tabular data. Therefore, these methods (as well as own preprocessing steps and how these are captured) are covered here in this section.

Here, we only show two preprocessing steps that are applied to datasets in `preprocess_datasets`(loaded in 2.2), which can also be performed individually depending upon requirement :
- Perform `Standard Preprocessing functions`.
- `Transform the dataset into an anomaly dataset`

In [10]:
#ID 11(3)                            SCALING APPLIED

#used imports from #ID 2(1)                                          
for dataset_name in datasets:
    
    
    datasets[dataset_name].treat_missing_values()
    datasets[dataset_name].normalize_columns()
    datasets[dataset_name].delete_duplicates()
    operations=datasets[dataset_name].operations_performed
    dataset_info_store(dataset_name,new_recipe_path,info_type='standard_functions',content=operations) 
   


#print("preprocesing performed on datasets! ")    
#print(datasets)

The file <b>f"{time}-{benchmark_name}-{benchmark_type}-recipe.yaml"</b> now contains information about how to preprocess(i.e. perform scaling) the file 

In [11]:
#ID 12(3)

!cat {new_recipe_path}

myTabularDataset:
- dataset
- standard_functions:
  - name: treat_missing_values
    parameters:
      missing_value: np.nan
      delete_attributes: true
  - name: normalize_columns
    parameters:
      cols_to_normalize:
  - name: delete_duplicates
    parameters: {}
spambase:
- dataset
- standard_functions:
  - name: treat_missing_values
    parameters:
      missing_value: np.nan
      delete_attributes: true
  - name: normalize_columns
    parameters:
      cols_to_normalize:
  - name: delete_duplicates
    parameters: {}


In [12]:
#ID 13(3)                            ANOMALY-DATASET CONVERSION PERFORMED

#used import from #ID 2 


#recipe_path=f"{benchmark_name}/{time}_{benchmark_name}_recipe.yaml"                                           

datasets_ad={}    
    # for storing dataset objects converted to anomaly-dataset
for dataset_name in datasets:   
    
     datasets_ad[dataset_name]= UnsupervisedAnomalyDataset(classification_dataset=datasets[dataset_name],
                                                       normal_labels=0)  
   
     normal_labels=datasets_ad[dataset_name].normal_labels 
     dataset_info_store(dataset_name,new_recipe_path,info_type='anomaly_dataset',content=normal_labels)   
                                                                            
print("datasets after adding anomaly-conversion datasets: ")    
print(datasets_ad)

datasets after adding anomaly-conversion datasets: 
{'myTabularDataset': <oab.data.unsupervised.UnsupervisedAnomalyDataset object at 0x7fc1833914d0>, 'spambase': <oab.data.unsupervised.UnsupervisedAnomalyDataset object at 0x7fc183457890>}


In [15]:
#ID 14(3)

!cat {new_recipe_path}

myTabularDataset:
- dataset
- standard_functions:
  - name: treat_missing_values
    parameters:
      missing_value: np.nan
      delete_attributes: true
  - name: normalize_columns
    parameters:
      cols_to_normalize:
  - name: delete_duplicates
    parameters: {}
- anomaly_dataset:
    arguments:
      normal_labels:
      - 0
      anomaly_labels:
spambase:
- dataset
- standard_functions:
  - name: treat_missing_values
    parameters:
      missing_value: np.nan
      delete_attributes: true
  - name: normalize_columns
    parameters:
      cols_to_normalize:
  - name: delete_duplicates
    parameters: {}
- anomaly_dataset:
    arguments:
      normal_labels:
      - 0
      anomaly_labels:


# **4. SAMPLING**

Here, we define the sampling parameters to sample from the datasets

In [13]:
#ID 15(4)

### ADD YOUR OWN SAMPLING PARAMETERS ###

# sampling parameters

n=25                          #Number of data points to sample                    
contamination_rate = 0.05         #Contamination rate when sampling, defaults to 0.05
n_steps = 4        # n_steps=10  #Number of samples to take, i.e., number of times sampling is repeated, defaults to 10 


#These below are the possible sampling types to sample from datasets
sampling_types=['unsupervised_multiple','unsupervised_single','unsupervised_multiple_benchmark']


sampling_type='unsupervised_multiple'  #by default for this run

sampling_params_current_run=[{'n':n,'contamination_rate':contamination_rate,'n_steps':n_steps},sampling_type]

sampling=[{sampling_type:sampling_params_current_run[0]}]
print(sampling)

#storing sampling info to recipe
for dataset_name in datasets_ad:
    dataset_info_store(dataset_name,new_recipe_path,'sampling',content=sampling)
 

[{'unsupervised_multiple': {'n': 25, 'contamination_rate': 0.05, 'n_steps': 4}}]


The above sampling parameters are utilized in
<b>#ID 24 '''LID 1''' and #ID 31 '''LID 2'''</b> 
for sampling the datasets(except the pre-installed mv_tec_ad_datasets) before training the algorithms.

In [14]:
# ID 16(4)
!cat {new_recipe_path}

myTabularDataset:
- dataset
- standard_functions:
  - name: treat_missing_values
    parameters:
      missing_value: np.nan
      delete_attributes: true
  - name: normalize_columns
    parameters:
      cols_to_normalize:
  - name: delete_duplicates
    parameters: {}
- anomaly_dataset:
    arguments:
      normal_labels:
      - 0
      anomaly_labels:
- sampling:
    unsupervised_multiple:
      n: 25
      contamination_rate: 0.05
      n_steps: 4
spambase:
- dataset
- standard_functions:
  - name: treat_missing_values
    parameters:
      missing_value: np.nan
      delete_attributes: true
  - name: normalize_columns
    parameters:
      cols_to_normalize:
  - name: delete_duplicates
    parameters: {}
- anomaly_dataset:
    arguments:
      normal_labels:
      - 0
      anomaly_labels:
- sampling:
    unsupervised_multiple:
      n: 25
      contamination_rate: 0.05
      n_steps: 4


Now, we will associate sampling information with each dataset loaded in the benchmark run:

In [15]:
#ID 17(4)
benchmarking_datasets={}

for (x,y) in datasets_ad.items():
    benchmarking_datasets[x]=[y,sampling_params_current_run]


print(benchmarking_datasets)    

{'myTabularDataset': [<oab.data.unsupervised.UnsupervisedAnomalyDataset object at 0x7fc1833914d0>, [{'n': 25, 'contamination_rate': 0.05, 'n_steps': 4}, 'unsupervised_multiple']], 'spambase': [<oab.data.unsupervised.UnsupervisedAnomalyDataset object at 0x7fc183457890>, [{'n': 25, 'contamination_rate': 0.05, 'n_steps': 4}, 'unsupervised_multiple']]}


The above dictionary <b>benchmarking_datasets</b> will be used for the Benchmarking as it contains all the information:"
    
    
    1.dataset_name
    2.final_dataset_object(preprocessed and anomaly-converted)
    3.sampling_info



# **5. ALGORITHM TRAINING AND TESTING**

We first download and import algorithms used for anomaly decection.

In [16]:
#ID 18(5)




# import anomaly detection algorithms from pyod
from pyod.models.knn import KNN 
from pyod.models.lof import LOF
from pyod.models.iforest import IForest
from pyod.models.abod import ABOD
from pyod.models.auto_encoder import AutoEncoder

import wget
wget.download('https://raw.githubusercontent.com/ISDM-CAU-Kiel/oab/master/notebooks/benchmark_tabular/ae_lof.py', f"{recipes_parent_path}/ae_lof.py")
from notebooks.benchmark_tabular.ae_lof import AELOF


### ADD your algo import here ###


100% [............................................................] 2486 / 2486

Firstly, we define hyperparameters for all algorithms and choose for benchmarking:

In [17]:
#ID 19(5)

  
### Extend Algos dictionary with own algorithm specifications as shown below for OAB algorithms ###   

### ADD YOUR OWN (HYPER)PARAMETERS AND THEIR VALUES FOR PRE-INSTALLED ALGOS###

# KNN
knn_factor = 0.05
knn_minimum = 10


# LOF
lof_factor = 0.1
lof_minimum = 10

# ABOD
abod_factor = 0.01
abod_minimum = 10



lst_benchmark_algorithms =[
   {   
       "algo_module_name": "pyod.models.knn" , 
       "algo_class_name": "KNN",
       "algo_name_in_result_table": "KNN",
       "algo_parameters": {'n_neighbors': {'knn_factor':0.05 ,'knn_minimum':10 }},
       "fit": {'method_name': 'fit', 'params': {}}, 
       "decision_scores": {'field_name': 'decision_scores_'}
    }   
    

]
''' ### uncomment to use these algos below for benchmarking ###
       ,
        
        {
       "algo_module_name": "pyod.models.iforest",   
       "algo_class_name": "IForest",
       "algo_name_in_result_table": "IForest",    
       "algo_parameters": {'random_state': 42} ,
        "fit": {'method_name': 'fit', 'params': {}}, 
        "decision_scores": {'field_name': 'decision_scores_'}
        },
        
        {
       "algo_module_name": "pyod.models.auto_encoder"   ,
       "algo_class_name": "AutoEncoder",
       "algo_name_in_result_table": "AutoEncoder",
       "algo_parameters":   {'hidden_neurons':[6,3,3,6], 'random_state': 42, 'verbose': 0},
        "fit": {'method_name': 'fit', 'params': {}}, 
       "decision_scores": {'field_name': 'decision_scores_'}
       } ,
       {   
       "algo_module_name": "pyod.models.abod" , 
       "algo_class_name": "ABOD",
       "algo_name_in_result_table": "ABOD", 
       "algo_parameters": {'n_neighbors':{'abod_factor':00.1 ,'abod_minimum':10 }}, 
       "fit": {'method_name': 'fit', 'params': {}}, 
       "decision_scores": {'field_name': 'decision_scores_'}
    } 
    ,
    
    {   
       "algo_module_name": "pyod.models.lof" , 
       "algo_class_name": "LOF",
       "algo_name_in_result_table": "LOF",
       "algo_parameters":  {'n_neighbors': {'lof_factor':0.1 ,'lof_minimum':10 }},
       "fit": {'method_name': 'fit', 'params': {}}, 
       "decision_scores": {'field_name': 'decision_scores_'}
    }
    ,
    
    {   
       "algo_module_name": "aelof" , 
       "algo_class_name": "AELOF",
       "algo_name_in_result_table": "AELOF", 
       "algo_parameters": {'AE_parameters':{'hidden_neurons':[6,3,3,6], 'random_state': 42, 'verbose': 0},
                            'LOF_parameters' : {'n_neighbors': {'lof_factor':0.1 ,'lof_minimum':10 }},
                           'random_state': 42},
    
       "fit": {'method_name': 'fit', 'params': {}}, 
       "decision_scores": {'field_name': 'decision_scores_'}
    }
    
    
    
]    
    
'''

### ADD OWN ALGORITHMS NAME(S) with algorithm specifications as shown above for OAB algorithms ###   

own_algorithms=[]  #add to this list e.g. { "algo_module_name": "own_algo" , "algo_class_name": "ownAlgoClass",.........."decision_scores": 'decision_scores_'} 

lst_benchmark_algorithms.extend(own_algorithms)


#seed defined for ths benchmark run for obtaining consistent results 
seed=42

<b>LOAD YOUR RECIPE</b> to be repdroduced and use it in the current benchmark run, lets,see  how it looks!

In [18]:
#ID 20(5)


### ADD AN OPTIONAL RECIPE  PATH TO ADD TO THIS BENCHMARK RUN START ###   

# Note: recipes of type "unsupervised tabular(ust) " i.e. of the format: 
#               "timestamp-benchmark_name-ust-recipe.yaml"
# can only be used for benchmarking in this notebook.

recipe_path=recipes_parent_path/"Paper_B"/"20211206060200-Paper_B-ust-recipe.yaml"

### ADD AN OPTIONAL RECIPE  PATH TO ADD TO THIS BENCHMARK RUN END ###   
    
### UNCOMMENT ONLY IF NO NEW DATASETS WERE ADDED IN THE BENCHMARK EXCEPT FROM RECIPE START ###     

#benchmarking_datasets={}
#lst_benchmark_algorithms=[]

### UNCOMMENT ONLY IF NO NEW DATASETS WERE ADDED IN THE BENCHMARK EXCEPT FROM RECIPE END ###  

!cat {recipe_path} 


annthyroid:
- dataset
- standard_functions:
  - name: normalize_columns
    parameters:
      cols_to_normalize:
      - 0
      - 1
  - name: treat_missing_values
    parameters:
      missing_value: np.nan
      delete_attributes: true
  - name: delete_duplicates
    parameters: {}
- anomaly_dataset:
    arguments:
      normal_labels: 0
      anomaly_labels:
- sampling:
    unsupervised_multiple:
      n: 25
      contamination_rate: 0.05
      n_steps: 1
myTabularDataset2:
- dataset
- standard_functions:
  - name: normalize_columns
    parameters:
      cols_to_normalize:
      - 0
      - 1
  - name: treat_missing_values
    parameters:
      missing_value: np.nan
      delete_attributes: true
  - name: delete_duplicates
    parameters: {}
- anomaly_dataset:
    arguments:
      normal_labels: 0
      anomaly_labels:
- sampling:
    unsupervised_multiple:
      n: 25
      contamination_rate: 0.05
      n_steps: 1
pyod.models.abod:
- algo_name
- init:
    params:
      n_neighbors

In [19]:
#ID 21(5)    # Execute this cell only when you already have a recipe file  to load from 


recipe_algos=data_from_recipe('algos',recipe_path) # all algo names from recipe extracted
#print(f"recipe_algos:\n{recipe_algos}")

recipe_datasets=data_from_recipe('datasets',recipe_path) # all dataset info(anomaly dataset object/anomalydataset params/sampling params) is perfomed and obtained
#print(f"\nrecipe_datasets:\n{recipe_datasets}")
 
recipe_seed=data_from_recipe('seed',recipe_path)  # obtained seeds to feed in this benchmark 
seed=recipe_seed   # seed of current benchmark is overwritten by recipe seed


pyod.models.abod----


annthyroid------
standard/custom preprocessing performed!
transformed to anomaly dataset!

myTabularDataset2------
standard/custom preprocessing performed!
transformed to anomaly dataset!


In [20]:
#ID 22(5)    


# adding recipe_datasets to benchmarking_datasets
for dataset_name in recipe_datasets:
    benchmarking_datasets[dataset_name]=recipe_datasets[dataset_name][:2]
#print(f"benchmarking_datasets: {benchmarking_datasets}") 
                                         
#adding algos from recipe_algos to lst_benchmarking_algos
for algo in recipe_algos:
    lst_benchmark_algorithms.append(algo)
#print(lst_benchmark_algorithms)


print("\nDatasets obtained from recipe:")    
for dataset_name in recipe_datasets:
    print(dataset_name)


print("\nAlgos obtained from recipe:") 
for algo_name in recipe_algos:
    print(algo_name['algo_module_name'])
    
  
print("\nAll Datasets for this benchmark run:")    
for dataset_name in benchmarking_datasets:
    print(dataset_name)

    
 
print("\nAll algos for this benchmark run:")
for algo in lst_benchmark_algorithms:
    #print(algo)
    print(algo['algo_module_name'])






Datasets obtained from recipe:
annthyroid
myTabularDataset2

Algos obtained from recipe:
pyod.models.abod

All Datasets for this benchmark run:
myTabularDataset
spambase
annthyroid
myTabularDataset2

All algos for this benchmark run:
pyod.models.knn
pyod.models.abod


Now, For every benchmark dataset , we sample from that dataset to train the algorithms and then predict the outcomes for each dataset with each algortihm and then store results in a evaluation object, which is then added to the comparison object to show the final Benchmarking results

In [21]:
#ID 23(5)
co = ComparisonObject()


for dataset_name in list(benchmarking_datasets.keys()):
    print(f'-------{dataset_name}-------') 
   
    #print(mvtec_ad_own_datasets_list)
    for alg in lst_benchmark_algorithms:
        
        print("------"+alg["algo_module_name"])
        eval_obj = EvaluationObject(algorithm_name=alg["algo_name_in_result_table"])
        sampling_type=benchmarking_datasets[dataset_name][1][1]
        sampling_params=benchmarking_datasets[dataset_name][1][0]
        
        for (x,y),sample_config in sample_unsupervised(dataset_name,sampling_type,sampling_params,benchmarking_datasets[dataset_name][0]):
                torch.manual_seed(seed)
                random.seed(seed)
                tf.random.set_seed(seed)
                np.random.seed(seed) 
                os.environ['PYTHONHASHSEED'] = str(seed)
                torch.cuda.manual_seed(seed)
                torch.cuda.manual_seed_all(seed)
                torch.backends.cudnn.deterministic = True
                torch.backends.cudnn.benchmark = False
                torch.use_deterministic_algorithms(True)
                
                algorithm_params=algo_params_tabular(alg,l=len(x))  
                mod = __import__(alg["algo_module_name"], fromlist=[alg["algo_class_name"]])
                algo = getattr(mod, alg["algo_class_name"])(**algorithm_params)  
                print('.', end='') # update to see progress 
                getattr(algo,alg["fit"]["method_name"])(x, **alg["fit"]["params"])  # fitting algo
                pred= getattr(algo,alg["decision_scores"]["field_name"]) #decision scores returned
                eval_obj.add(ground_truth=y, prediction=pred, description=sample_config)  
        print("\n")    
        eval_desc = eval_obj.evaluate(print=True, metrics=['roc_auc', 'adjusted_average_precision', 'precision_recall_auc'])
        co.add_evaluation(eval_desc)
        print("\n")
      

-------myTabularDataset-------
------pyod.models.knn
....

Evaluation on dataset myTabularDataset with normal labels [0] and anomaly labels [1, 2].
Total of 4 datasets. Per dataset:
25 instances, contamination_rate 0.05.
Mean 	 Std_dev 	 Metric
0.726 	 0.156 		 roc_auc
0.279 	 0.203 		 adjusted_average_precision
0.258 	 0.201 		 precision_recall_auc


------pyod.models.abod
....

Evaluation on dataset myTabularDataset with normal labels [0] and anomaly labels [1, 2].
Total of 4 datasets. Per dataset:
25 instances, contamination_rate 0.05.
Mean 	 Std_dev 	 Metric
0.717 	 0.224 		 roc_auc
0.282 	 0.281 		 adjusted_average_precision
0.279 	 0.258 		 precision_recall_auc


-------spambase-------
------pyod.models.knn
....

Evaluation on dataset spambase with normal labels [0] and anomaly labels [1].
Total of 4 datasets. Per dataset:
25 instances, contamination_rate 0.05.
Mean 	 Std_dev 	 Metric
0.554 	 0.118 		 roc_auc
0.107 	 0.063 		 adjusted_average_precision
0.105 	 0.031 		 precision_

# **6. EVALUATION**

Here , we will see how different metrics can be selected when evaluating an algorithm's performance.

In previous section while creating an evalutation description,  we used all metrics for evaluation:

     eval_desc = eval_obj.evaluate(print=False, metrics=all_metrics)
    
    

In [22]:
#ID 24(6)

# to use a subset, first see which ones are available

print(all_metrics)

['roc_auc', 'average_precision', 'adjusted_average_precision', 'precision_n', 'adjusted_precision_n', 'precision_recall_auc']


In [23]:
#ID 25(6)

#### ADD YOUR OWN NUMBER OF METRICS ###

#Then we can  select an arbitrary subset
metrics=['roc_auc', 'precision_recall_auc']

# **7. SHOW BENCHMARK RESULTS**

We compare by printing, the results of the evaluations of different Algo-Dataset combinations.

\[Latex version: bold for highest, italics for second highest, ?\]

In [24]:
#ID 26(7)

# print results in easily readable format
co.print_results()

For roc_auc:
         myTabularDataset  spambase  annthyroid  myTabularDataset2   Average
KNN              0.725543  0.554348    0.402174           0.913043  0.648777
ABOD             0.717391  0.603261    0.673913           0.913043  0.726902
Average          0.721467  0.578804    0.538043           0.913043       NaN
For adjusted_average_precision:
         myTabularDataset  spambase  annthyroid  myTabularDataset2   Average
KNN              0.279282  0.106785    0.015802           0.637681  0.259887
ABOD             0.281834  0.192115    0.126553           0.637681  0.309546
Average          0.280558  0.149450    0.071177           0.637681       NaN
For precision_recall_auc:
         myTabularDataset  spambase  annthyroid  myTabularDataset2   Average
KNN              0.258499  0.105418    0.063936           0.633333  0.265297
ABOD             0.278528  0.209591    0.117445           0.633333  0.309724
Average          0.268514  0.157504    0.090690           0.633333       NaN


In [25]:
#ID 28(7)

# print results in easily readable format with standard deviations
co.print_results(include_stdevs=True)

For roc_auc:
        myTabularDataset      spambase    annthyroid myTabularDataset2  \
KNN         0.726+-0.156  0.554+-0.118  0.402+-0.000      0.913+-0.000   
ABOD        0.717+-0.224  0.603+-0.077  0.674+-0.000      0.913+-0.000   
Average            0.721         0.579         0.538             0.913   

          Average  
KNN      0.648777  
ABOD     0.726902  
Average       NaN  

For adjusted_average_precision:
        myTabularDataset      spambase    annthyroid myTabularDataset2  \
KNN         0.279+-0.203  0.107+-0.063  0.016+-0.000      0.638+-0.000   
ABOD        0.282+-0.281  0.192+-0.187  0.127+-0.000      0.638+-0.000   
Average            0.281         0.149         0.071             0.638   

          Average  
KNN      0.259887  
ABOD     0.309546  
Average       NaN  

For precision_recall_auc:
        myTabularDataset      spambase    annthyroid myTabularDataset2  \
KNN         0.258+-0.201  0.105+-0.031  0.064+-0.000      0.633+-0.000   
ABOD        0.279+-0.258 

In [26]:
# ID 29(7)

co.print_latex(include_stdevs=True)

For roc_auc:
\begin{center}
\begin{tabular}{  c c c c c c  }
  & myTabularDataset & spambase & annthyroid & myTabularDataset2 & Average \\
  KNN & \textbf{0.726$\pm$0.156} & \textit{0.554$\pm$0.118} & \textit{0.402$\pm$0.000} & \textbf{0.913$\pm$0.000} & \textit{0.649} \\
  ABOD & \textit{0.717$\pm$0.224} & \textbf{0.603$\pm$0.077} & \textbf{0.674$\pm$0.000} & \textit{0.913$\pm$0.000} & \textbf{0.727} \\
  Average & 0.721 & 0.579 & 0.538 & 0.913 &    \\
\end{tabular}
\end{center}

For adjusted_average_precision:
\begin{center}
\begin{tabular}{  c c c c c c  }
  & myTabularDataset & spambase & annthyroid & myTabularDataset2 & Average \\
  KNN & \textit{0.279$\pm$0.203} & \textit{0.107$\pm$0.063} & \textit{0.016$\pm$0.000} & \textbf{0.638$\pm$0.000} & \textit{0.260} \\
  ABOD & \textbf{0.282$\pm$0.281} & \textbf{0.192$\pm$0.187} & \textbf{0.127$\pm$0.000} & \textit{0.638$\pm$0.000} & \textbf{0.310} \\
  Average & 0.281 & 0.149 & 0.071 & 0.638 &    \\
\end{tabular}
\end{center}

For preci

# **8. REPRODUCIBILITY**

 ## **8.1 Creating recipes**

This section shows **how `oab` can be used to make sampling results easily reproducible** .
 

`yaml` files play an integral role in making reproducibility work, as they store the operations and parameters performed on the data.

We will see how to produce a recipe(.yaml) of the Benchmarkrun already performed  in <b>#ID 23(5)</b>

In <b>#ID 11(3) #ID 13(3) #ID 15(4)</b>,  We already performed operations on own datasets and OAB's datasets, and then already stored the daasets information as we can see below: 

In [27]:
#ID 30(8)
!cat {new_recipe_path}

myTabularDataset:
- dataset
- standard_functions:
  - name: treat_missing_values
    parameters:
      missing_value: np.nan
      delete_attributes: true
  - name: normalize_columns
    parameters:
      cols_to_normalize:
  - name: delete_duplicates
    parameters: {}
- anomaly_dataset:
    arguments:
      normal_labels:
      - 0
      anomaly_labels:
- sampling:
    unsupervised_multiple:
      n: 25
      contamination_rate: 0.05
      n_steps: 4
spambase:
- dataset
- standard_functions:
  - name: treat_missing_values
    parameters:
      missing_value: np.nan
      delete_attributes: true
  - name: normalize_columns
    parameters:
      cols_to_normalize:
  - name: delete_duplicates
    parameters: {}
- anomaly_dataset:
    arguments:
      normal_labels:
      - 0
      anomaly_labels:
- sampling:
    unsupervised_multiple:
      n: 25
      contamination_rate: 0.05
      n_steps: 4


Now, we will store the information of  datasets and algorithms information from <b>Paper_B's</b> recipe
and only of the algorithms of this benchmark in the new recipe:

In [28]:
#ID 31(8)
# adding datasets from recipe used in in benchmark run in #ID 23(5)
for dataset_name in recipe_datasets:
    
    
    #storing anomaly parameters
    dataset_info_store(dataset_name,new_recipe_path,info_type='anomaly_dataset',content=recipe_datasets[dataset_name][0].normal_labels)
    
    # storing preprocesing parameters
    dataset_info_store(dataset_name,new_recipe_path,info_type='standard_functions',content=recipe_datasets[dataset_name][3]) 
    #dataset_info_store(dataset_name,new_recipe_path,info_type='custom_functions',content=recipe_datasets[dataset_name][4]) 
    
    
    #storing sampling parameters
    if dataset_name[:9]!='mvtec_ad_':
      sampling_data=recipe_datasets[dataset_name][1]
      dataset_info_store(dataset_name,new_recipe_path,'sampling',content=sampling_data)
        

Now,we will store information about <b>Algorithms and their hyperparameters</b> in the recipe(.yaml) 

In [29]:
#ID 32(8)
for algo in lst_benchmark_algorithms:
    
    x=algo["algo_module_name"]
    y=['algo_name',
         
         {
         'init': 
          
               {

       'params':algo["algo_parameters"]
          
               },
        
        'fit':algo["fit"]   
        ,

        'decision_scores':algo["decision_scores"]
         },
         
         algo["algo_class_name"]
        
        ]
                 
     
    yaml=YAML(typ='rt')
    yaml_content = yaml.load(Path("./") / new_recipe_path)
    yaml_content[x]=y
    yaml_content['seed']=[seed]          # adding seed to new recipe
    yaml.dump(yaml_content, Path("./") /new_recipe_path)

In [30]:
#ID 33(8)
!cat {new_recipe_path}

myTabularDataset:
- dataset
- standard_functions:
  - name: treat_missing_values
    parameters:
      missing_value: np.nan
      delete_attributes: true
  - name: normalize_columns
    parameters:
      cols_to_normalize:
  - name: delete_duplicates
    parameters: {}
- anomaly_dataset:
    arguments:
      normal_labels:
      - 0
      anomaly_labels:
- sampling:
    unsupervised_multiple:
      n: 25
      contamination_rate: 0.05
      n_steps: 4
spambase:
- dataset
- standard_functions:
  - name: treat_missing_values
    parameters:
      missing_value: np.nan
      delete_attributes: true
  - name: normalize_columns
    parameters:
      cols_to_normalize:
  - name: delete_duplicates
    parameters: {}
- anomaly_dataset:
    arguments:
      normal_labels:
      - 0
      anomaly_labels:
- sampling:
    unsupervised_multiple:
      n: 25
      contamination_rate: 0.05
      n_steps: 4
annthyroid:
- dataset
- anomaly_dataset:
    arguments:
      normal_labels:
      - 0
      a

In **f"{time}-{benchmark_name}-{benchmark_type}-recipe.yaml"**, we now see the sampling parameters, anomaly- dataset-conversion parameters, hyperparamters along with the algorithms for "unsupervised_multiple_with_training_split". If sampling is done in a different scenario, e.g., unsupervised multiple, this would also be stored in f"{benchmark_name}/{time}_{benchmark_name}_recipe.yaml" using a different key in the sampling dict.



### 2. Reproducing the experiment

To reproduce the recipe created in the previous section,
we refer to <b>Section 5 #ID 20(5)</b> where we can reproduce the run as well as extend benchmarks!

# **9. EXTEND EXISTING BENCHMARK(own algorithm)**

To extend the existing benchmark here basically means to add  our own algorithm to the benchmark and to show the comparison results of pre-installed algorithms while also loading our own dataset.


1. We load the datasets. To know how to do that, we can refer to  **Section "1. Data" and "2. Data Selection"**
2. Then, load own algorithm as we will see in the next sub-section.

## **9.1 Loading own Algorithm**

In this subsection, you will see **how an own unsupervised anomaly detection algorithm** can easily be used within oab to be evaluated. We will see how a class representing an algorithm can be structured and how its performance is evaluated.

Of course, this is not the only way to use the functionality provided by oab. We do consider it to be the simplest way however.

In [44]:
#ID 34(9)
# download example algorithm and inspect content
import wget
wget.download('https://raw.githubusercontent.com/jandeller/test/main/RandomGuesser.py',f"{recipes_parent_path}/RandomGuesser.py")
!cat {recipes_parent_path}/RandomGuesser.py

100% [..............................................................] 204 / 204import numpy as np

class RandomGuesser():

    def fit(self, X):
        "Assign a random number to each sample"
        n_samples = X.shape[0]
        self.decision_scores_ = np.random.randn(n_samples)


The sample `RandomGuesser` algorithm shown here is - as the name suggests - a random guesser, i.e., it assigns random anomaly scores to the samples.

An algorithm used for unsupervised anomaly detection needs to specify a `fit(X_train)` method for training and a `decision_scores_` variable for inference that returns an anomaly score per data point in the test set.

It is of course possible to rename the method and field, use a method for accessing the anomaly scores, etc. Note that if this is done, the following code has to be changed accordingly. Adhering to the conventions described above (`fit(X_train)` and `decision_scores_`) allows you to use the same interface as algorithms from [`PyOD`](https://pyod.readthedocs.io/en/latest/) as shown when [comparing algorithms using `oab`](https://colab.research.google.com/drive/1aV_itaYCJgzdZ1lQ7SUyHQ7z01xSPxDN?usp=sharing#scrollTo=QnAfCGTGL7xv).

In [45]:
#ID 35(9)
# used imports from #ID 3(0),#ID 18(5)
#used sampling parameters from #ID 15(4)

# and import the RandomGuesser
from notebooks.benchmark_tabular.RandomGuesser import RandomGuesser

   
own_algorithms=[{
    
       ### ADD YOUR OWN ALGO DETAILS IN THIS FORM ###
       "algo_module_name": "RandomGuesserSemisupervised",   
       "algo_class_name": "RandomGuesserSemisupervised",
       "algo_name_in_result_table": "RandomGuesserSemisupervised",
       "algo_parameters": {},
        "fit": {'method_name': 'fit', 'params': {}}, 
        "decision_scores": {'field_name': 'decision_scores_'}
        }]


The `own_algorithms` list in the above cell #ID 38 can be added to lst_benchmarking_algos as mentioned in #ID 19(5) to use this algorithm in a benchmark run shown in #ID 24(5) along with other algorithms

In [39]:
#ID 36(9)

#  A comparison object is created for comparing the evaluations of different Algo-Dataset combinations
co = ComparisonObject()

for dataset_name in benchmarking_datasets:
  print(f"-------{dataset_name}-------")
  eval_obj = EvaluationObject(algorithm_name="RandomGuesser")
  print("-------RandomGuesser")  
  for (x,y), settings in benchmarking_datasets[dataset_name][0].sample_multiple(n=n, 
                                                                  contamination_rate=contamination_rate, 
                                                                 n_steps=n_steps):
      print(".", end=" ") # update to see progress
      rg = RandomGuesser()
      rg.fit(x) # data is fitted to RandomGuesser
      pred = rg.decision_scores_ # and decision_scores_ is accessed
      eval_obj.add(y, pred, settings)
  print("\n")
  eval_desc = eval_obj.evaluate(metrics=['roc_auc', 'adjusted_average_precision', 'precision_recall_auc'])
  # added to comparison object
  co.add_evaluation(eval_desc)
  print("\n")



-------myTabularDataset-------
-------RandomGuesser
. . . . 

Evaluation on dataset myTabularDataset with normal labels [0] and anomaly labels [1, 2].
Total of 4 datasets. Per dataset:
25 instances, contamination_rate 0.05.
Mean 	 Std_dev 	 Metric
0.663 	 0.141 		 roc_auc
0.326 	 0.207 		 adjusted_average_precision
0.337 	 0.216 		 precision_recall_auc


-------spambase-------
-------RandomGuesser
. . . . 

Evaluation on dataset spambase with normal labels [0] and anomaly labels [1].
Total of 4 datasets. Per dataset:
25 instances, contamination_rate 0.05.
Mean 	 Std_dev 	 Metric
0.500 	 0.212 		 roc_auc
0.162 	 0.215 		 adjusted_average_precision
0.193 	 0.207 		 precision_recall_auc


-------annthyroid-------
-------RandomGuesser
. . . . 

Evaluation on dataset annthyroid with normal labels [0] and anomaly labels [1.0].
Total of 4 datasets. Per dataset:
25 instances, contamination_rate 0.05.
Mean 	 Std_dev 	 Metric
0.636 	 0.134 		 roc_auc
0.126 	 0.052 		 adjusted_average_precision
0

As in the above code, We store the evaluations of our own algorithm in evaluation object which is then added to comparison object.Similarly, we can create evaluation objects for other algorithms and add them to comparison object for final benchmarking  as shown in Section 5

Finally, we show below the benchmarking results of our algorithm as described in "**Section 7. Show Benchmarking Results**"

In [40]:
#ID 37(9)

# print results in easily readable format
co.print_results()

For roc_auc:
               myTabularDataset  spambase  annthyroid  myTabularDataset2  \
KNN                    0.725543  0.554348    0.402174           0.913043   
ABOD                   0.717391  0.603261    0.673913           0.913043   
RandomGuesser          0.663043  0.500000    0.635870           0.663043   
Average                0.701993  0.552536    0.570652           0.829710   

                Average  
KNN            0.648777  
ABOD           0.726902  
RandomGuesser  0.615489  
Average             NaN  
For adjusted_average_precision:
               myTabularDataset  spambase  annthyroid  myTabularDataset2  \
KNN                    0.279282  0.106785    0.015802           0.637681   
ABOD                   0.281834  0.192115    0.126553           0.637681   
RandomGuesser          0.325997  0.162094    0.125700           0.325997   
Average                0.295704  0.153664    0.089352           0.533786   

                Average  
KNN            0.259887  
ABOD       

In [41]:
#ID 38(9)
# print results in easily readable format with standard deviations
co.print_results(include_stdevs=True)

For roc_auc:
              myTabularDataset      spambase    annthyroid myTabularDataset2  \
KNN               0.726+-0.156  0.554+-0.118  0.402+-0.000      0.913+-0.000   
ABOD              0.717+-0.224  0.603+-0.077  0.674+-0.000      0.913+-0.000   
RandomGuesser     0.663+-0.141  0.500+-0.212  0.636+-0.134      0.663+-0.141   
Average                  0.702         0.553         0.571             0.830   

                Average  
KNN            0.648777  
ABOD           0.726902  
RandomGuesser  0.615489  
Average             NaN  

For adjusted_average_precision:
              myTabularDataset      spambase    annthyroid myTabularDataset2  \
KNN               0.279+-0.203  0.107+-0.063  0.016+-0.000      0.638+-0.000   
ABOD              0.282+-0.281  0.192+-0.187  0.127+-0.000      0.638+-0.000   
RandomGuesser     0.326+-0.207  0.162+-0.215  0.126+-0.052      0.326+-0.207   
Average                  0.296         0.154         0.089             0.534   

                Averag

In [42]:
#ID 39(9)

co.print_latex(include_stdevs=True)

For roc_auc:
\begin{center}
\begin{tabular}{  c c c c c c  }
  & myTabularDataset & spambase & annthyroid & myTabularDataset2 & Average \\
  KNN & \textbf{0.726$\pm$0.156} & \textit{0.554$\pm$0.118} & 0.402$\pm$0.000 & \textbf{0.913$\pm$0.000} & \textit{0.649} \\
  ABOD & \textit{0.717$\pm$0.224} & \textbf{0.603$\pm$0.077} & \textbf{0.674$\pm$0.000} & \textit{0.913$\pm$0.000} & \textbf{0.727} \\
  RandomGuesser & 0.663$\pm$0.141 & 0.500$\pm$0.212 & \textit{0.636$\pm$0.134} & 0.663$\pm$0.141 & 0.615 \\
  Average & 0.702 & 0.553 & 0.571 & 0.830 &    \\
\end{tabular}
\end{center}

For adjusted_average_precision:
\begin{center}
\begin{tabular}{  c c c c c c  }
  & myTabularDataset & spambase & annthyroid & myTabularDataset2 & Average \\
  KNN & 0.279$\pm$0.203 & 0.107$\pm$0.063 & 0.016$\pm$0.000 & \textbf{0.638$\pm$0.000} & \textit{0.260} \\
  ABOD & \textit{0.282$\pm$0.281} & \textbf{0.192$\pm$0.187} & \textbf{0.127$\pm$0.000} & \textit{0.638$\pm$0.000} & \textbf{0.310} \\
  RandomGuesser

So,This was our example algorithm. Other algorithms can be used to run and extend benchmarks,  Please refer  to <b>#ID 19(5)</b>.